In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
import cv2
from PIL import Image
from shapely.geometry import Polygon

from utils.poisson import PoissonDisc
from utils.coords import *
from utils.divtree_reader import readDivideTree
from utils.meshwriter import writeTerrainMesh
from synthesis.divtree_to_dem import *

%matplotlib inline

mpl.rcParams['image.cmap'] = 'terrain'

In [ ]:
# read divide tree
terrainName = 'synthesis_pyrenees'

peakCoords, peakElevs, saddleCoords, saddleElevs, saddlePeaks, _ = readDivideTree(terrainName + '_dividetree.txt')
peakCoords   = deg2km(peakCoords)
saddleCoords = deg2km(saddleCoords)

In [ ]:
# Terrain size, we could also compute the bbox of the peak/saddle positions in the divide tree
terrainSize = [90, 90]

# Minimum elevation value we want this terrain to have
minTerrainElev = 0.5*saddleElevs.min()

In [ ]:
print(terrainSize)
print(peakCoords.max())
print(peakElevs.size, saddleElevs.size)
print(peakElevs.max(), peakElevs.min(), saddleElevs.min(), minTerrainElev)

In [ ]:
# Poisson samples precomputation, takes a while. We could store them in a file

# We usually set poissonRadius = 1.5*refineDistance (see later in reconsParams)
poissonRadius  = 0.18 # km

poissonSamples = PoissonDisc(width=terrainSize[0], height=terrainSize[1], r=poissonRadius, k=15).sample()
poissonSamples = np.array([[s[0], s[1]] for s in poissonSamples])

In [ ]:
# reconstruction algorithm parameters (in practice, we only tune maxSlopeCoeff, if any)
reconsParams = {
    'minTerrainElev': minTerrainElev, # clip elevations lower than this
    'maxSlopeCoeff': 0.5,             # linear factor for the river slope, lower = gentler slopes
    
    'refineDistance': 0.12,           # resampling distance for fine river/ridge networks (affects performance)
    'riversPerturbation': 0.20,       # planar perturbation of new river nodes, as % of edge length
    'ridgesPerturbation': 0.15,       # planar perturbation of new ridge nodes, as % of ridge length
    
    'useDrainageArea': True,          # river width dependant on drainage area: w ~ A^0.4
    'maxRiverWidth': 0.3,             # max river (flat terrain) width, as % of distance between river and ridge
    
    'coarseRiverSmoothIters': 4,      # smoothing of the coarse river nodes elevation
    'refinedRiverSmoothIters': 5,     # smoothing of the fine river nodes elevation
    'refinedRiverSmoothPosIters': 1   # smoothing of the fine river nodes position
}

In [ ]:
np.random.seed(42)

meshVerts, meshElevs, meshTris, debugInfo = divideTreeToMesh(peakCoords, peakElevs, saddleCoords, saddleElevs, saddlePeaks, 
                                                             terrainSize, poissonSamples, reconsParams)
print('Done!')

# Output

### Mesh

In [ ]:
# Geometry as mesh, exact representation of the terrain we synthesized, 
# it contains all peaks and saddles and the elevation is exact

In [ ]:
xMax = terrainSize[0]
yMax = terrainSize[1]
bbox = Polygon([[0, 0], [0, yMax], [xMax, yMax], [xMax, 0]])

writeTerrainMesh(terrainName + '_mesh.obj', meshVerts.copy(), np.maximum(0, meshElevs), meshTris.copy(), bbox)

print('Mesh elevation range', meshElevs.max(), meshElevs.min())

### Raster heightfield

In [ ]:
# Function provided as debug visualization of the DEM
# Introduces small errors due to pixel resolution and elevation discretization.
# Also, we use scipy delaunay because it contains useful functions that speed up the code,
# but unlike Triangle library it's unconstrained and might omit some (small) ridge segments thus creating saddles/peaks.
# However, since we already sampled points over the whole domain and we include the Steiner points, should be almost equal.
# For best results, directly rasterize the geometry mesh obtained above.

def heightfield_delaunay(coords, elevs, terrainSize, hfsize):
    #https://codereview.stackexchange.com/questions/41024/faster-computation-of-barycentric-coordinates-for-many-points
    
    # pixel coords
    x = np.linspace(0, 1, hfsize[0] + 1)[:-1] + 0.5/hfsize[0]
    y = np.linspace(0, 1, hfsize[1] + 1)[:-1] + 0.5/hfsize[1]
    xv, yv = np.meshgrid(x, y)
    pixcoords = np.array([xv.flatten(), yv.flatten()]).T

    # compute Delaunay, and find the triangle containing each target (-1 if not found)
    pointCoords = np.concatenate([coords/terrainSize, np.array([[0, 0], [0, 1], [1, 1], [0, 0]])])
    pointElevs  = np.concatenate([elevs,  np.array([0, 0, 0, 0])])
    pointElevs  = pointElevs[:, np.newaxis]
    delaunayTri = Delaunay(pointCoords)
    triangles   = delaunayTri.find_simplex(pixcoords)

    # compute barycentric coordinates
    X = delaunayTri.transform[triangles,:2]
    Y = pixcoords - delaunayTri.transform[triangles,2]
    b = np.einsum('...ij,...j->...i', X, Y) # multiply and sum last dimension of X with last dimension of Y
    bcoords = np.c_[b, 1 - b.sum(axis=1)]

    # interpolate elevations
    ielevs = np.einsum('ij,ijk->ik', bcoords, pointElevs[delaunayTri.simplices[triangles]])

    # store result
    pixels = (hfsize * pixcoords).astype(int)
    hfield = np.zeros(hfsize)
    hfield[pixels[:,0], pixels[:,1]] = ielevs.flatten()
    return hfield


In [ ]:
pixelMeters = 30 # m

hfsize = (np.array(terrainSize)*1000/pixelMeters).astype(int)
print(hfsize)

In [ ]:
hf = heightfield_delaunay(meshVerts, meshElevs, terrainSize, hfsize)
hf = np.rot90(hf)
print(hf.min(), hf.max())

cv2.imwrite(terrainName + '_dem16.png', np.maximum(10*hf, 0).astype(np.uint16))

### Distance fields

In [ ]:
# distance fields to use in the erosion and enhancement part of the algorithm
dfPeaks = np.ones(hfsize)
for p in peakCoords:
    pi = (p/terrainSize*hfsize).astype(int)
    cv2.circle(dfPeaks, (pi[1], pi[0]), 2, color=0, thickness=-1)
dfPeaks = cv2.distanceTransform(dfPeaks.astype(np.uint8), cv2.DIST_L2, cv2.DIST_MASK_PRECISE)
dfPeaks = np.minimum(dfPeaks, 255.0)

dfSaddles = np.ones(hfsize)
for s in saddleCoords:
    pi = (s/terrainSize*hfsize).astype(int)
    cv2.circle(dfSaddles, (pi[1], pi[0]), 2, color=0, thickness=-1)
dfSaddles = cv2.distanceTransform(dfSaddles.astype(np.uint8), cv2.DIST_L2, cv2.DIST_MASK_PRECISE)
dfSaddles = np.minimum(dfSaddles, 255.0)

dfRidges = np.ones(hfsize)
ridgeCoords = meshVerts[:debugInfo['numRidgeVerts'], :]
for seg in debugInfo['ridgeSegments']:
    rfrom,rto = seg
    p1 = (ridgeCoords[rfrom,:]/terrainSize*hfsize).astype(int)
    p2 = (ridgeCoords[rto,:]/terrainSize*hfsize).astype(int)
    cv2.line(dfRidges, (p1[1], p1[0]), (p2[1], p2[0]), color=0, thickness=1)    
dfRidges = cv2.distanceTransform(dfRidges.astype(np.uint8), cv2.DIST_L2, cv2.DIST_MASK_PRECISE)
dfRidges = np.minimum(dfRidges, 255.0)

imgDF = np.rot90(np.dstack([dfRidges, dfSaddles, dfPeaks]))
_ = cv2.imwrite(terrainName + '_distfield.png', imgDF.astype(np.uint8))

# Debug visualizations

In [ ]:
# change these to crop a portion of the terrain, e.g. for figures
xmin = ymin = 0
xmax = terrainSize[0]
ymax = terrainSize[1]

In [ ]:
# Voronoi cells and river lines

fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(111)


# ridgelines from divide tree
for i in range(saddlePeaks.shape[0]):
    p1 = peakCoords[saddlePeaks[i,0]]
    p2 = peakCoords[saddlePeaks[i,1]]
    ps = saddleCoords[i]
    ax.plot([p1[0], ps[0]], [p1[1], ps[1]], color='orange', linewidth=3, zorder=1)
    ax.plot([p2[0], ps[0]], [p2[1], ps[1]], color='orange', linewidth=3, zorder=1)

# voronoi cells
voronoiCells = debugInfo['voronoiRegions']
voronoiVerts = debugInfo['voronoiVerts']
for ip,poly in enumerate(voronoiCells):    
    for i,p in enumerate(poly):
        p1 = voronoiVerts[p]
        p2 = voronoiVerts[poly[(i+1)%len(poly)]]
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], color='lightgray', linewidth=1)

# river lines
riverLines = debugInfo['coarseRiverLines']
for line in riverLines:    
    for i,p in enumerate(line.coords):
        p1 = p
        p2 = line.coords[(i+1)%len(line.coords)]
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], color='steelblue', linewidth=2)
        
     
_ = plt.xlim(xmin-10, xmax+10)
_ = plt.ylim(ymin-10, ymax+10)        


#ax.set_axis_off()
#ax.get_xaxis().set_visible(False)
#ax.get_yaxis().set_visible(False)
#plt.axis('off')  
#plt.savefig('voronoi.pdf', dpi=300, bbox_inches='tight', pad_inches=0)

In [ ]:
# coarse river network

fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(111)


# ridgelines from divide tree
for i in range(saddlePeaks.shape[0]):
    p1 = peakCoords[saddlePeaks[i,0]]
    p2 = peakCoords[saddlePeaks[i,1]]
    ps = saddleCoords[i]
    ax.plot([p1[0], ps[0]], [p1[1], ps[1]], color='orange', linewidth=3, zorder=1)
    ax.plot([p2[0], ps[0]], [p2[1], ps[1]], color='orange', linewidth=3, zorder=1)

# river flow and drainage area
voronoiVerts = debugInfo['voronoiVerts']
riverDrainArea = debugInfo['coarseRiverDrainArea']
riverFlowTo = debugInfo['coarseRiverFlowTo']
for rfrom,rto in enumerate(riverFlowTo):
    if rto >= 0:
        p1 = voronoiVerts[rfrom,:]
        p2 = voronoiVerts[rto,:]
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], color='steelblue', linewidth=0.15*riverDrainArea[rto]**0.4)
        #ax.arrow(p1[0], p1[1], p2[0] - p1[0], p2[1] - p1[1], color='blue', linewidth=0.01, length_includes_head=True)

# river sources
riverSources = debugInfo['coarseRiverSources']
riverElevs   = debugInfo['coarseRiverElevs']
for rs in riverSources:
    if not rs in riverFlowTo:
        p1 = voronoiVerts[rs,:]
        ax.scatter(p1[0], p1[1], marker='o', s=20, color='purple', zorder=3)
        #ax.annotate('%.2f'%(riverElevs[rs]), xy=np.clip(p1, 0, terrainSize), fontsize=5)
        #ax.annotate('%d'%(rs), xy=np.clip(p1, 0, terrainSize), fontsize=5)
        
    
_ = plt.xlim(xmin, xmax)
_ = plt.ylim(ymin, ymax)


#ax.set_axis_off()
#ax.get_xaxis().set_visible(False)
#ax.get_yaxis().set_visible(False)
#plt.axis('off')  
#plt.savefig('coarseRivers.pdf', dpi=300, bbox_inches='tight', pad_inches=0)

In [ ]:
# refined networks

drawPoisson = False
drawMesh = False


fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(111)


numRidgeVerts = debugInfo['numRidgeVerts']
numRiverVerts = debugInfo['numRiverVerts']
numPoissonVerts = debugInfo['numPoissonVerts']

# refined ridgelines
ridgeCoords = meshVerts[:numRidgeVerts,:]
ridgeSegs   = debugInfo['ridgeSegments']
for seg in ridgeSegs:
    rfrom,rto = seg
    p1 = ridgeCoords[rfrom,:]
    p2 = ridgeCoords[rto,:]
    ax.plot([p1[0], p2[0]], [p1[1], p2[1]], color='orange', linewidth=3, zorder=1)
                
              
# river flow and drainage area
riverCoords = meshVerts[numRidgeVerts:numRidgeVerts+numRiverVerts,:]
riverDrainArea = debugInfo['riverDrainArea']
riverFlowTo = debugInfo['riverFlowTo']
for rfrom,rto in enumerate(riverFlowTo):
    if rto >= 0:
        p1 = riverCoords[rfrom,:]
        p2 = riverCoords[rto,:]
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], color='steelblue', linewidth=0.15*riverDrainArea[rto]**0.4)
        #ax.arrow(p1[0], p1[1], p2[0] - p1[0], p2[1] - p1[1], color='blue', linewidth=0.01, length_includes_head=True)
                        
# Poisson samples
poissonCoords = meshVerts[numRidgeVerts+numRiverVerts:numRidgeVerts+numRiverVerts+numPoissonVerts,:]
if drawPoisson:
    ax.scatter(poissonCoords[:,0], poissonCoords[:,1], marker='o', s=5, color='purple')
    
# Mesh triangles
if drawMesh:
    ax.triplot(meshVerts[:,0], meshVerts[:,1], meshTris, color='black', linewidth=0.2, zorder=5)  
            
                     
_ = plt.xlim(xmin, xmax)
_ = plt.ylim(ymin, ymax)


#ax.set_axis_off()
#ax.get_xaxis().set_visible(False)
#ax.get_yaxis().set_visible(False)
#plt.axis('off')  
#plt.savefig('refinedNetworks.pdf', dpi=300, bbox_inches='tight', pad_inches=0)

In [ ]:
# Poisson samples closest river and ridge

numRidgeVerts = debugInfo['numRidgeVerts']
numRiverVerts = debugInfo['numRiverVerts']
numPoissonVerts = debugInfo['numPoissonVerts']
ridgeCoords = meshVerts[:numRidgeVerts,:]
riverCoords = meshVerts[numRidgeVerts:numRidgeVerts+numRiverVerts,:]
poissonCoords = meshVerts[numRidgeVerts+numRiverVerts:numRidgeVerts+numRiverVerts+numPoissonVerts,:]

closestRidgeIdx = debugInfo['closestRidgeIdx']
closestRiverIdx = debugInfo['closestRiverIdx']


fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(111)


ax.scatter(ridgeCoords[:,0], ridgeCoords[:,1], marker='o', color='orange', s=10)
ax.scatter(riverCoords[:,0], riverCoords[:,1], marker='o', color='steelblue', s=10)

for test in np.random.permutation(numPoissonVerts)[:10]:
    p1 = ridgeCoords[closestRidgeIdx[test]]
    p2 = riverCoords[closestRiverIdx[test]]
    ps = poissonCoords[test]

    ax.plot([p1[0], ps[0]], [p1[1], ps[1]], color='darkgreen', linewidth=3)
    ax.plot([p2[0], ps[0]], [p2[1], ps[1]], color='darkgreen', linewidth=3)
    ax.scatter(ps[0], ps[1], marker='o', color='orange', s=100)
    ax.scatter(p1[0], p1[1], marker='o', color='red', s=100)
    ax.scatter(p2[0], p2[1], marker='o', color='blue', s=100)


_ = plt.xlim(xmin, xmax)
_ = plt.ylim(ymin, ymax)
